
# Logging in Python: Observability and Design

In production-grade software, your code is rarely run under your direct supervision. Instead, it operates within remote systems, embedded services, or cloud environments—often out of sight and in constant motion. In such contexts,

When an error happens, you need to have a log that you or a software tester can capture to troubleshoot. 

This means using the standard output (ie. Print command) will not be useful for you or your users during runtime. So far you have been logging detailed information about internal application state, warnings, and errors that happen during program execution. 

> "If code is what tells the system what to do, then logging is how the system tells *you* what it did."

## Why Logging Matters

Consider this:
- You're not watching your program run.
- Failures might only be discovered hours or days after they occur.
- You need post-mortem forensic capabilities.

Thus, logging isn't a luxury; it is your first line of **introspection, debugging, and accountability**.

### Standard Output ≠ Logging

While `print()` statements may help during early development, they fall short in reliability and configurability:
- Cannot be easily turned off/on.
- Cannot be redirected, filtered, or structured.
- Do not convey severity or source.

Python comes with a built-in `logging` module that provides a basic yet powerful logging system. 

### Keywords
Each log message is associated with a **severity level**, providing a mechanism to filter and prioritize information. These are:

| Level      | Description |
|------------|-------------|
| `CRITICAL` / `FATAL` | System is unusable. Cannot continue. |
| `ERROR`    | Major failure. An operation failed. |
| `WARNING`  | Anomalies that are recoverable, but notable. |
| `INFO`     | Confirmation that things are working as expected. |
| `DEBUG`    | Diagnostic output useful during development. |
| `NOTSET`   | Captures everything (used for internal fallback). |

In [ ]:
import logging

logging.basicConfig()

# Create the logger
debuglog = logging.getLogger("systemConfiguredLogger")

debuglog.log(logging.CRITICAL, "OMG THE WORLD IS ENDING!!!!")

debuglog.critical("there is no more disk space")
debuglog.error("file not found creating new file")
debuglog.warning("disk space is low")
debuglog.info("user logged in with correct password")
#notice how info is not displayed below
#The default configuration captures messages up to the logging level of 
#WARNING level and outputs them to standard output


ERROR:systemConfiguredLogger:OMG THE WORLD IS ENDING!!!!
CRITICAL:systemConfiguredLogger:there is no more disk space
ERROR:systemConfiguredLogger:file not found creating new file


In [6]:
import logging

logging.basicConfig(level=logging.DEBUG, format='%(levelname)s: %(message)s')

# Create the logger
debuglog = logging.getLogger("my_logger")

debuglog.log(logging.CRITICAL, "OMG THE WORLD IS ENDING!!!!")

debuglog.critical("there is no more disk space")
debuglog.error("file not found creating new file")
debuglog.warning("disk space is low")
debuglog.info("user logged in with correct password")


CRITICAL: OMG THE WORLD IS ENDING!!!!
CRITICAL: there is no more disk space
ERROR: file not found creating new file
INFO: user logged in with correct password


## Wait They look the same

Because Jupyter notebooks persist state, if you configure logging once, it will retain the configuration. To reconfigure cleanly:

```python
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
```

Or simply use:

```python
logging.basicConfig(..., force=True)
```

In [7]:
# Remove all existing handlers
# for handler in logging.root.handlers[:]:
#     logging.root.removeHandler(handler)

#Note:`force=True` (available in Python 3.8+):
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s',force=True)


### Loging types
there a 3 built in by default log handlers they are Null, StreamHandler, FileHandler

**null** - This handler does nothing

**StreamHandler** - This handler emits messages to a given stream

**FileHandler** - emits messages to the file specified by the filename argument

### Logging handlers 
There is a long list of handelers we will not go over all of them. 

In [8]:
# FileHandler - timedRotating

import logging.handlers
from datetime import timedelta, datetime

logger = logging.getLogger()

logger.setLevel(logging.INFO) # set the logging level to INFO

# create a file handler that logs to a file
logger.addHandler(
    logging.handlers.TimedRotatingFileHandler(
        filename="archived_log.log", #file name
        when="D", #rolls the log every day
        backupCount=3, # only keep 3 days worth of file backups
    )
)


## Gu Loves his freezegun
    Nothing is better then a freeze-gun 
The freezegun package is used to trick Python into thinking that time is passing at an increased pace and is very useful in testing

In [ ]:
!pip install freezegun

In [11]:

from datetime import timedelta, datetime
import time
import logging
import freezegun

logger = logging.getLogger() #use the logger created above

def main():
    with freezegun.freeze_time() as frozen:
        for _ in range(10):
            frozen.tick(timedelta(hours=24))
            time.sleep(0.1)
            logger.info(f"all your base are belong to us at {datetime.now()}")

if __name__ == "__main__":
    main()

INFO: all your base are belong to us at 2025-04-09 23:59:50.825668
INFO: all your base are belong to us at 2025-04-10 23:59:50.825668
INFO: all your base are belong to us at 2025-04-11 23:59:50.825668
INFO: all your base are belong to us at 2025-04-12 23:59:50.825668
INFO: all your base are belong to us at 2025-04-13 23:59:50.825668
INFO: all your base are belong to us at 2025-04-14 23:59:50.825668
INFO: all your base are belong to us at 2025-04-15 23:59:50.825668
INFO: all your base are belong to us at 2025-04-16 23:59:50.825668
INFO: all your base are belong to us at 2025-04-17 23:59:50.825668
INFO: all your base are belong to us at 2025-04-18 23:59:50.825668


In [12]:
!cat archived_log.log*
#windows
#!type archived_log.log*


all your base are belong to us at 2025-04-18 23:59:50.825668
all your base are belong to us at 2025-04-15 23:59:50.825668
all your base are belong to us at 2025-04-16 23:59:50.825668
all your base are belong to us at 2025-04-17 23:59:50.825668


In [13]:
!rm archived_log.log*
#Windows
#!del archived_log.log*

## Formatting  

Notice that the above log does not have the "INFO" tag and that is because it was not configured. 


Lets correct this by using the formater 

In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO) #log Level
formatter = logging.Formatter(
    fmt=(
        "%(asctime)s | %(levelname)s | "
        "%(message)s"
    )
)
handler = logging.handlers.TimedRotatingFileHandler(
    filename="formatedLog.log",
    when="D",
    backupCount=3,
)
handler.setFormatter(formatter)
logger.addHandler(handler)

*Formater is customizable see below:*

**fmt** - This is a string formatting pattern for the output message. 

**datefmt** - This is a date formatting pattern for representing the message timestamp. 

**style** - This specifies the string formatting style used for the fmt argument. It can be either '%' (percent formatting), '{' (str.format() formatting), or '$' (string.Template formatting). It defaults to '%'.

**validate** - This specifies whether to validate the fmt formatting argument against the style argument. It defaults to True.

In [6]:
from datetime import timedelta, datetime
import time
import logging
import freezegun
def main():
    with freezegun.freeze_time() as frozen:
        for _ in range(5):
            frozen.tick(timedelta(hours=24))
            time.sleep(0.1)
            for _ in range(100):
                logger.warning(f"all your base are belong to us")
                logger.critical(f"all your base are belong to us")
if __name__ == "__main__":
    main()

In [3]:
!cat formatedLog.log

2025-04-13 17:08:04,992 | WARNING | all your base are belong to us


### Costomized Logging
in your adventures you may find that some libraries log differently then what you desire. 

```python
logger = logging.getLogger(__name__)

debuglog = logging.getLogger("my_logger")
```
\_\_name\_\_" will be the named log



In [ ]:
# silance the log
logger.disabled = False
#Overriding handlers
logger.handlers.clear()
#Changing logger verbosity
logger.setLevel(logging.WARNING)

### Fun with ~flags~ logs 

Periods are an import part of logs. A period will build a hierarchy of loggers.

for example  logger.lib.util

The system will create/search for logger registered under logger.lib.util

Then will create/search for logger regestered as logger.lib then logger. 

note if logger.util or logger are not found logger.lib.util will become the parrent. 

Best practice is to define log handlers only at the top-level root logger. Otherwise, it will be hard to track all propagation rules and make sure that every message is logged only once



In [ ]:
parrentLog = logging.getLogger("my_logger")
parrentLog.setLevel(logging.INFO)
#parrentLog.setLevel(logging.CRITICAL)
childLog = logging.getLogger("my_logger.util")
childLog.setLevel(logging.CRITICAL)
#childLog.setLevel(logging.INFO)
formatter = logging.Formatter(
    fmt=(
        "%(asctime)s | %(levelname)s | "
        "%(message)s"
    )
)
handler = logging.handlers.TimedRotatingFileHandler(
    filename="parent.log",
    when="D",
    backupCount=3,
)
handler2 = logging.handlers.TimedRotatingFileHandler(
    filename="child.log",
    when="D",
    backupCount=3,
)
handler.setFormatter(formatter)
parrentLog.addHandler(handler)
#childLog.addHandler(handler2)

In [ ]:
childLog.critical(f"all your base are belong to us")
childLog.info(f"hello")

In [ ]:
!cat child.log
!cat parent.log